In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import citypy module to determine city name from coordinates
from citipy import citipy
# import datetime
from datetime import datetime
import requests
from config import weather_api_key
import time
# Import linear regression dependency
from scipy.stats import linregress

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs 
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create a list to hold city names
cities = []

# Identify the cities for each longitude and latitude combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If city is unique we will add it to the cities list
    if city not in cities:
        cities.append(city)
# Print city count to confirm
print(len(cities))


629


In [ ]:
city_data = []
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Print the beginning of the logging
print('Beginning data retrieval    ')
print('----------------------------')

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
     # Run an API request for each of the cities.
   # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Weather Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")

Beginning data retrieval    
----------------------------
Processing Record 1 of Set 1 | mount gambier
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | north bend
Processing Record 4 of Set 1 | carolina
Processing Record 5 of Set 1 | mandali
Processing Record 6 of Set 1 | port alfred
Processing Record 7 of Set 1 | nanakuli
Processing Record 8 of Set 1 | french harbor
Processing Record 9 of Set 1 | vila velha
Processing Record 10 of Set 1 | eastbourne
Processing Record 11 of Set 1 | provideniya
Processing Record 12 of Set 1 | hermanus
Processing Record 13 of Set 1 | hasaki
